In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [8]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from category_encoders import CatBoostEncoder

In [9]:
aux = pd.read_csv("andy/train.csv");

In [10]:
df = aux.copy()

In [11]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [12]:
df.dtypes

id                              int64
titulo                         object
descripcion                    object
tipodepropiedad                object
direccion                      object
ciudad                         object
provincia                      object
antiguedad                    float64
habitaciones                  float64
garages                       float64
banos                         float64
metroscubiertos               float64
metrostotales                 float64
idzona                        float64
lat                           float64
lng                           float64
fecha                          object
gimnasio                      float64
usosmultiples                 float64
piscina                       float64
escuelascercanas              float64
centroscomercialescercanos    float64
precio                        float64
dtype: object

In [13]:
df.garages.fillna(0, inplace = True)
df.gimnasio.fillna(0, inplace = True)
df.usosmultiples.fillna(0, inplace = True)
df.piscina.fillna(0, inplace = True)
df.escuelascercanas.fillna(0, inplace = True)
df.centroscomercialescercanos.fillna(0, inplace = True)

In [14]:
df.drop(columns = 'id', inplace = True)
#Saco el id porque es unico para cada propriedad

In [15]:
#Para la antiguedad relleno con el promedio de las 5 que mas aparecen
aux1 = df.antiguedad.value_counts().head(5).reset_index()
aux1['cantidad'] = aux1.antiguedad
aux1['valores'] = aux1['index']*aux1['cantidad']
df.antiguedad.fillna(aux1.valores.sum()/aux1.cantidad.sum(), inplace = True)
df.antiguedad = df.antiguedad.astype(int)

In [16]:
#Reemplazo las habitaciones por las que mas aparecen
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

In [17]:
#Reemplazo los banos por 1 que es lo minimo posible
df.banos.fillna(1, inplace=True)

In [18]:
#Para los metros totales relleno con 0 al principio y hago el swap
df.metrostotales.fillna(value='0', inplace = True)
df.metroscubiertos.fillna(value='0', inplace = True)
df.metrostotales = df.metrostotales.astype(int)
df.metroscubiertos = df.metroscubiertos.astype(int)
df['metrostotales'] = df[['metrostotales', 'metroscubiertos']].max(axis=1)
df['metroscubiertos'] = df[['metrostotales', 'metroscubiertos']].min(axis=1)

In [19]:
aux2 = df[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'mean'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]

In [20]:
df.idzona.fillna(df.ciudad.map(ids), inplace = True)
df.idzona.fillna(0, inplace=True)

In [21]:
df.drop(['lat', 'lng', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad', 'provincia', 'fecha'], axis=1, inplace=True)
df.columns

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'precio'],
      dtype='object')

In [22]:
features = df.drop('precio', axis=1).columns
X_train,X_test,y_train,y_test = train_test_split(df[features], df.precio,test_size=0.3, random_state = 0)

In [23]:
clf = RandomForestClassifier(n_estimators = 40, max_depth = 8)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
for feature in sorted(zip( clf.feature_importances_,features)):
    print(feature)

(0.013239626166054513, 'escuelascercanas')
(0.014249473621152368, 'usosmultiples')
(0.014624145092732197, 'centroscomercialescercanos')
(0.016961621726130895, 'piscina')
(0.017820738744678107, 'gimnasio')
(0.058037126166861644, 'garages')
(0.07127512544246205, 'antiguedad')
(0.07352984719675389, 'habitaciones')
(0.10678644370086318, 'banos')
(0.17585153141523224, 'idzona')
(0.18160344379423923, 'metrostotales')
(0.2560208769328397, 'metroscubiertos')


In [25]:
preds = clf.predict(X_test)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, preds)

In [ ]:
x = df.drop(columns = ['id', 'titulo', 'descripcion', 'direccion', 'ciudad', 'lat', 'lng']).copy()

In [ ]:
x.metrostotales.fillna(value='0', inplace = True)
x.metroscubiertos.fillna(value='0', inplace = True)
x.metrostotales = x.metrostotales.astype(int)
x.metroscubiertos = x.metroscubiertos.astype(int)
x['metrostotales'] = x[['metrostotales', 'metroscubiertos']].max(axis=1)
x['metroscubiertos'] = x[['metrostotales', 'metroscubiertos']].min(axis=1)
x['patio'] = x['metrostotales'] - x['metroscubiertos']

In [ ]:
x['fecha'] = pd.to_datetime(x['fecha'])
x['anio'] = x.fecha.dt.year
x['mes'] = x.fecha.dt.month
x['dia'] = x.fecha.dt.day
x = x.drop(columns = 'fecha')

In [ ]:
transformador_numeros = SimpleImputer(fill_value = 0)
#transformador_catboost = Pipeline(steps=[('imputer1', SimpleImputer(strategy='most_frequent')),
#                                           ('catboost', ce.CatBoostEncoder(handle_unknown='ignore'))])
transformador_onehot = Pipeline(steps=[('imputer2', SimpleImputer(strategy='most_frequent')),
                                        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
cols_numericas = ['antiguedad', 'habitaciones','garages', 'banos', 'metroscubiertos',
                  'metrostotales', 'idzona', 'gimnasio', 'usosmultiples',
                  'piscina', 'escuelascercanas','centroscomercialescercanos', 'patio', 'anio', 'mes', 'dia']
#cols_catboost = ['ciudad']
cols_onehot = ['tipodepropiedad', 'provincia']

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, cols_numericas),
#        ('cat', transformador_catboost, cols_catboost),
        ('one', transformador_onehot, cols_onehot)
    ])

In [ ]:
features = x.drop('precio', axis=1).columns
X_train,X_test,y_train,y_test = train_test_split(x[features], x.precio,test_size=0.3, random_state = 0)
X_train.columns

In [ ]:
clf = RandomForestClassifier(n_estimators=10000, random_state=0)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', clf)
                             ])

In [ ]:
my_pipeline.fit(X_train,y_train)
#X_train.columns

In [ ]:
a = SimpleImputer()
x[cols_numericas] = a.fit_transform(x[cols_numericas])

In [ ]:
x.dtypes

In [29]:
aux.ciudad.isna().sum()

372

In [30]:
aux.idzona.isna().sum()

28621

In [32]:
aux.provincia.value_counts()

Distrito Federal         58790
Edo. de México           41607
Jalisco                  21238
Querétaro                16988
Nuevo León               15324
Puebla                   10421
San luis Potosí           8447
Yucatán                   7928
Morelos                   7337
Veracruz                  5762
Quintana Roo              4756
Chihuahua                 4590
Coahuila                  3695
Baja California Norte     3220
Sonora                    2988
Guanajuato                2860
Guerrero                  2678
Hidalgo                   2521
Michoacán                 2471
Tamaulipas                2303
Durango                   2275
Sinaloa                   1806
Aguascalientes            1753
Baja California Sur       1700
Nayarit                   1352
Chiapas                   1126
Colima                    1008
Tabasco                    994
Tlaxcala                   839
Oaxaca                     711
Campeche                   263
Zacatecas                   94
Name: pr

In [37]:
aux.head(15)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0
5,126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.0,2.0,1.0,...,23650.0,19.300612,-99.148475,2014-03-18 00:00:00,0.0,0.0,0.0,0.0,1.0,1100000.0
6,139233,de oportunidad casa en san lorenzo,"ubicada en esquina, pertenece san lorenzo agen...",Casa,NaN,Oaxaca de Juárez,Oaxaca,NaN,3.0,1.0,...,73510.0,17.143504,-96.803504,2016-02-23 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0
7,5013,casa emilia en venta en selvamar playa del carmen,casa emilia en venta playa del carmenfracciona...,Casa,condominio el trebol,Playa del Carmen,Quintana Roo,2.0,4.0,2.0,...,130510.0,20.672612,-87.037968,2016-10-20 00:00:00,0.0,0.0,0.0,0.0,0.0,4200000.0
8,44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.0,2.0,1.0,...,9010.0,NaN,NaN,2014-01-06 00:00:00,0.0,0.0,0.0,1.0,1.0,310000.0
9,134537,terreno,"terreno de 5.500m2 bardeado, uso de suelo h-20...",Terreno,Av. Morelos,Ixtapaluca,Edo. de México,NaN,NaN,NaN,...,59171.0,19.316000,-98.887000,2016-12-22 00:00:00,0.0,0.0,0.0,0.0,0.0,6200000.0


In [ ]:
aux.direccion.nunique().sum()